# Seting Up Djanig

In [1]:
import setup

setup.init_django()

# Imports

In [2]:
from rag import (
    db as rag_db,
    engines as rag_engines,
    settings as rag_settings,
    updaters as rag_updaters,
    sync as rag_sync, 
    patches as rag_patches,
)

from IPython.display import Markdown, display

In [3]:
from typing import Optional, Union
from sqlalchemy import create_engine, text
from llama_index.core.tools import QueryEngineTool

# Connect to database

In [4]:
rag_settings.init()
rag_db.init_vector_db()

Sync is necessary only when the new data added

In [ ]:
# rag_sync.full_sync()

# Define the tools

In [5]:
semantic_query_engine = rag_engines.get_semantic_query_engine("Product")  
sql_query_engine = rag_engines.get_sql_query_engine()

In [6]:
print(rag_settings.VECTOR_DB_NAME, rag_settings.VECTOR_DB_TABLE_NAMES)

vector_db ['employeeroles', 'employees', 'producttypes', 'products', 'inventoryitems', 'productinventoryrequirements']


In [7]:
vector_tool = QueryEngineTool.from_defaults(
    query_engine=semantic_query_engine,
    description=(
        "Useful for answering semantic questions about coffee shop operations, "
        "including products, employees, inventory, and recipes"
    ),
)


In [8]:
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    description=(
        "Useful for translating a natural language query into a SQL query over "
        "tables containing:EmployeeRole, Employee, ProductType, Product, InventoryItem, ProductInventoryRequirement."
    ),
)

In [9]:
query_engine = rag_patches.MySQLAutoVectorQueryEngine(
    sql_tool,
    vector_tool,
)

# Text2SQL

In [12]:
response = query_engine.query(
    "give me the list of barista"
)

Querying other query engine: The question 'give me the list of barista' is related to coffee shop operations and employees, which is covered by choice (2).


In [13]:
display(Markdown(response.response))

I'm afraid I can't provide a list of baristas based on the information available.

In [27]:
response = query_engine.query(
    "لیست تمام کارکنان"
)

Querying SQL database: The query 'لیست تمام کارکنان' translates to 'list all employees,' which is relevant to translating a natural language query into a SQL query over tables containing EmployeeRole and Employee.
SQL query: SELECT coffeshop_employee.name, coffeshop_employee.hire_date, coffeshop_employee.hourly_rate, coffeshop_employee.is_active, coffeshop_employeerole.name AS role_name
FROM coffeshop_employee
JOIN coffeshop_employeerole ON coffeshop_employee.role_id = coffeshop_employeerole.id
ORDER BY coffeshop_employee.name;
SQL response: Here is the list of all employees in the coffee shop:

1. **David Wilson**
   - **Hire Date:** 2024-04-26
   - **Hourly Rate:** $18.00
   - **Role:** Inventory Specialist
   - **Status:** Active

2. **Emily Davis**
   - **Hire Date:** 2024-10-17
   - **Hourly Rate:** $13.50
   - **Role:** Waiter
   - **Status:** Active

3. **Harry Smith**
   - **Hire Date:** 2022-03-05
   - **Hourly Rate:** $35.00
   - **Role:** Manager
   - **Status:** Active

4. 

In [28]:
display(Markdown(response.response))

Here is the list of all employees in the coffee shop:

1. **David Wilson**
   - **Hire Date:** 2024-04-26
   - **Hourly Rate:** $18.00
   - **Role:** Inventory Specialist
   - **Status:** Active

2. **Emily Davis**
   - **Hire Date:** 2024-10-17
   - **Hourly Rate:** $13.50
   - **Role:** Waiter
   - **Status:** Active

3. **Harry Smith**
   - **Hire Date:** 2022-03-05
   - **Hourly Rate:** $35.00
   - **Role:** Manager
   - **Status:** Active

4. **Jane Smith**
   - **Hire Date:** 2024-04-02
   - **Hourly Rate:** $15.00
   - **Role:** Barista
   - **Status:** Active

5. **Jasmine Lee**
   - **Hire Date:** 2025-02-26
   - **Hourly Rate:** $15.00
   - **Role:** Kitchen Staff
   - **Status:** Active

6. **John Doe**
   - **Hire Date:** 2024-03-26
   - **Hourly Rate:** $15.00
   - **Role:** Barista
   - **Status:** Active

7. **Michael Johnson**
   - **Hire Date:** 2024-05-26
   - **Hourly Rate:** $16.00
   - **Role:** Kitchen Staff
   - **Status:** Active

8. **Robert Brown**
   - **Hire Date:** 2025-01-07
   - **Hourly Rate:** $14.00
   - **Role:** Cashier
   - **Status:** Active

All employees are currently active.

In [25]:
response = query_engine.query(
    "averge price of all products"
)

Querying SQL database: The first choice mentions 'Product' and 'ProductType', which are relevant to determining the average price of all products.
SQL query: SELECT AVG(coffeshop_product.price) AS average_price
FROM coffeshop_product;
SQL response: The average price of all products is $3.56.
Transformed query given SQL response: None


In [26]:
display(Markdown(response.response))

The average price of all products is $3.56.